In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**MAIN**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from IPython.display import HTML

plt.rc('font', family='NanumGothic')

file_path = "/content/drive/MyDrive/Colab Notebooks/01 DATA/accidentInfoList.CSV"
df = pd.read_csv(file_path, encoding='cp949')

In [ ]:
#  미사용 피처 삭제
df.drop(columns=['사고번호'], inplace=True)
# '사고일시' 열을 변환
df['사고일시'] = pd.to_datetime(df['사고일시'], format='%Y년 %m월 %d일 %H시')
# '시군구' 피처에서 '구' 이전의 정보만 남기기
df['시군구'] = df['시군구'].apply(lambda x: x.split(' ')[1])\
# ECLO 피처 생성
df['ECLO'] = df['사망자수'] * 10 + df['중상자수'] * 5 + df['경상자수'] * 3 + df['부상신고자수'] * 1

# 결측치 처리
df['피해운전자 차종'].fillna('NaN', inplace=True)
df['피해운전자 성별'].fillna('NaN', inplace=True)
df['피해운전자 상해정도'].fillna('상해없음', inplace=True)


In [ ]:
# 숫자 추출 함수 생성
def extract_numbers_from_age(age):
    numbers = re.findall(r'\d+', str(age))
    if numbers:
        return int(numbers[0])
    else:
        return np.nan

# 문자 데이터 제거 및 숫자만 추출
df['가해운전자 연령'] = df['가해운전자 연령'].apply(extract_numbers_from_age)
df['피해운전자 연령'] = df['피해운전자 연령'].apply(extract_numbers_from_age)

# 결측치를 평균으로 대체
mean_age_driver = round(df['가해운전자 연령'].mean(), 1)
mean_age_victim = round(df['피해운전자 연령'].mean(), 1)

df['가해운전자 연령'].fillna(mean_age_driver, inplace=True)
df['피해운전자 연령'].fillna(mean_age_victim, inplace=True)

**SUB**

In [ ]:
#기초통계분석
basic_stats = df.describe(include='all', datetime_is_numeric=True)
html = basic_stats.to_html()
html = html.replace('class="table table-striped"', 'class="table table-bordered table-striped"')
display(HTML(html))


In [ ]:
print(df.columns.tolist())

['사고일시', '요일', '시군구', '사고내용', '사망자수', '중상자수', '경상자수', '부상신고자수', '사고유형', '법규위반', '노면상태', '기상상태', '도로형태', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', 'ECLO']


In [ ]:
print(df.head())

In [ ]:
# 원핫 인코딩 예시
categorical_features = ['요일']
encoded_df = pd.get_dummies(df[categorical_features])
df = df.drop(columns=categorical_features)
df = pd.concat([df, encoded_df], axis=1)


In [ ]:
#전반적인 프로세스

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

file_path = "/content/drive/MyDrive/Colab Notebooks/01 DATA/accidentInfoList.CSV"
df = pd.read_csv(file_path, encoding='cp949')

#범주형 데이터 처리
categorical_features = ['요일']
encoded_df = pd.get_dummies(df[categorical_features])
df = df.drop(columns=categorical_features)
df = pd.concat([df, encoded_df], axis=1)

# 피처 엔지니어링
df['ECLO'] = df['사망자수'] * 10 + df['중상자수'] * 5 + df['경상자수'] * 3 + df['부상신고자수'] * 1

# 필요한 피처 및 타겟 데이터 선택
X = df[['요일_금요일', '요일_목요일', '요일_수요일', '요일_월요일', '요일_일요일', '요일_토요일', '요일_화요일']]
y = df['ECLO']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 회귀 모델 생성 및 훈련
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 모델 평가
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)


In [ ]:
#임시 저장
output_file_path = "/content/drive/MyDrive/Colab Notebooks/01 DATA/modified_accidentInfoList.csv"
df.to_csv(output_file_path, index=False)